# **[Кейс «SQL-запрос»](https://netology.ru/profile/program/job-tz-an-3/lessons/210465/lesson_items/1112134)**

## **Задание**

Вы — аналитик данных. Руководитель дал вам задание поработать с таблицей logs действий пользователей (user_id, event, event_time, value). Действия пользователей поделены на сессии - последовательности событий, в которых между соседними по времени событиями промежуток не более 5 минут. Т.е. длина всей сессии может быть гораздо больше 5 минут, но между каждыми последовательными событиями не должно быть более 5 минут.
Поле event может принимать разные значения, в том числе ’template_selected’ (пользователь выбрал некий шаблон). В случае, если event=’template_selected’, то в value записано название этого шаблона (например, ’pop_art_style’).

### **Задача**   
Напишите SQL-запрос, выводящий 5 шаблонов, которые чаще всего применяются юзерами 2 и более раза подряд в течение одной сессии.

Этапы выполнения задания:

* Выполните задачу кейса.
* Укажите в работе ваши контакты и объем времени, затраченного на выполнение кейса.
* Загрузите данные на github.
* Прикрепите ссылку на работу в окошке «Решение».

In [1]:
import random
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta
from sqlalchemy import create_engine

### **Случайным образом сгенерируем датасет для следующих условий:**   
   * 500 пользователей
   * в течение 72 часов
   * участвуют в от 1 до 10 сессях, т.е. последовательности event'ов, между которыми проходит не более 5 минут
   * каждая из сессий состоит из 1 до 50 event'ов 20 типов value
   
### **Для простоты будем считать, что event = 0 это выбор шаблона**

In [2]:
user_id_list = np.random.randint(4000, 5000, 500)
user_sessions = {}
for user_id in user_id_list:
    sessions = {}
    for session in range(np.random.randint(1, 11)):
        events = {}
        time_start = np.random.randint(1, 259201)
        for event in range(np.random.randint(1, 51)):
            event_time = datetime.strptime('01.01.2022 00:00:00', '%d.%m.%Y %H:%M:%S') + timedelta(seconds = np.random.randint(1, 299) + time_start)
            event_value = np.random.randint(1, 21)
            events[event] = [event_value, event_time]
        sessions[session] = events
    user_sessions[user_id] = sessions
log = pd.DataFrame.from_dict({(i,j,k): user_sessions[i][j][k]
        for i in user_sessions.keys()
        for j in user_sessions[i].keys()
        for k in user_sessions[i][j].keys()},
        orient='index')
log = log.reset_index()
log[['user_id', 'session', 'event']] = pd.DataFrame(log['index'].tolist(), index=log.index)
log = log.drop('index', axis = 1)
log = log.rename ({0: 'value', 1: 'event_time'}, axis = 1)
log = log.reindex(columns = ['user_id', 'session', 'event', 'event_time', 'value'])
log = log.sort_values('event_time').drop('session', axis = 1).reset_index()
log = log.drop('index', axis = 1)

In [3]:
log

,user_id,event,event_time,value
0,4698,13,2022-01-01 00:00:22,1
1,4698,14,2022-01-01 00:00:36,11
2,4698,16,2022-01-01 00:00:42,13
3,4698,19,2022-01-01 00:00:46,13
4,4698,28,2022-01-01 00:00:58,9
...,...,...,...,...
56180,4929,2,2022-01-04 00:02:56,8
56181,4929,20,2022-01-04 00:02:58,11
56182,4929,8,2022-01-04 00:03:10,19
56183,4929,22,2022-01-04 00:03:16,7


In [5]:
#установление соединения с локальной БД
engine = create_engine('sqlite:///test_project.db', echo=True)
sqlite_connection = engine.connect()

In [6]:
sqlite_table = 'logs_5020'
log.to_sql(sqlite_table, sqlite_connection, if_exists='fail')

2022-09-29 07:31:17,937 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("logs_5020")
2022-09-29 07:31:17,940 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-29 07:31:17,943 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("logs_5020")
2022-09-29 07:31:17,944 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-09-29 07:31:17,949 INFO sqlalchemy.engine.Engine 
CREATE TABLE logs_5020 (
	"index" BIGINT, 
	user_id BIGINT, 
	event BIGINT, 
	event_time DATETIME, 
	value BIGINT
)


2022-09-29 07:31:17,950 INFO sqlalchemy.engine.Engine [no key 0.00160s] ()
2022-09-29 07:31:17,956 INFO sqlalchemy.engine.Engine COMMIT
2022-09-29 07:31:17,958 INFO sqlalchemy.engine.Engine CREATE INDEX ix_logs_5020_index ON logs_5020 ("index")
2022-09-29 07:31:17,960 INFO sqlalchemy.engine.Engine [no key 0.00166s] ()
2022-09-29 07:31:17,964 INFO sqlalchemy.engine.Engine COMMIT
2022-09-29 07:31:18,036 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-09-29 07:31:18,387 INFO sqlalchemy.engine.Engine INSERT 